In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.image as mpimg
import random
import seaborn as sns
import scipy.signal as ss
from scipy.special import i0
from scipy.ndimage import convolve1d
from matplotlib import animation
from matplotlib import rc as matrc
matrc('animation', html='jshtml', embed_limit=20971520*2)

In [ ]:
def pooling(mat,ksize,method='max',pad=False):
    '''Non-overlapping pooling on 2D or 3D data.

    <mat>: ndarray, input array to pool.
    <ksize>: tuple of 2, kernel size in (ky, kx).
    <method>: str, 'max for max-pooling, 
                   'mean' for mean-pooling.
    <pad>: bool, pad <mat> or not. If no pad, output has size
           n//f, n being <mat> size, f being kernel size.
           if pad, output has size ceil(n/f).

    Return <result>: pooled matrix.
    '''

    m, n = mat.shape[:2]
    ky,kx=ksize

    _ceil=lambda x,y: int(np.ceil(x/float(y)))

    if pad:
        ny=_ceil(m,ky)
        nx=_ceil(n,kx)
        size=(ny*ky, nx*kx)+mat.shape[2:]
        mat_pad=np.full(size,np.nan)
        mat_pad[:m,:n,...]=mat
    else:
        ny=m//ky
        nx=n//kx
        mat_pad=mat[:ny*ky, :nx*kx, ...]

    new_shape=(ny,ky,nx,kx)+mat.shape[2:]

    if method=='max':
        result=np.nanmax(mat_pad.reshape(new_shape),axis=(1,3))
    else:
        result=np.nanmean(mat_pad.reshape(new_shape),axis=(1,3))

    return result

In [ ]:
# Constructing the Gabor filters

def gabor2D_Lee(size, orientation, sigma_deno, spat_freq):
    num_orient = len(orientation)
    
    # Generate 2d grid of a Gabor wavelet
    x_arr = np.arange(size[0]) - (size[0]+1)/2 + 1
    y_arr = np.arange(size[1]) - (size[1]+1)/2 + 1

    # Generate a 4D Tensor to hold all the Gabor filters
    x_arr = np.arange(size[0]) - (size[0]+1)/2 + 1
    y_arr = np.arange(size[1]) - (size[1]+1)/2 + 1
    x_arr = np.tile(x_arr, size[0]) # horizontal steps
    y_arr = np.repeat(y_arr, size[1]) # vertical steps
    gabor_arr_cos = np.zeros((size[0], size[0], num_orient))
    gabor_arr_sin = np.zeros((size[0], size[0], num_orient))

    # Generate Gabor wavelets
    for i in range(num_orient):
        gabor_arr_cos[:,:,i] = np.reshape(gaborFunc_Lee(x_arr, y_arr, size, orientation[i], sigma_deno, spat_freq)[0], (size[0], size[1]))
        gabor_arr_sin[:,:,i] = np.reshape(gaborFunc_Lee(x_arr, y_arr, size, orientation[i], sigma_deno, spat_freq)[1], (size[0], size[1]))

    return [gabor_arr_cos, gabor_arr_sin]

In [ ]:
def gaborFunc_Lee(x, y, size, theta, sigma_deno, spat_freq):
    # Gaussian envelope of Gabor function
    sigma = size[0]/sigma_deno

    gaussEnv = 4 * (x * np.cos(theta) + y * np.sin(theta))**2 + (-x * np.sin(theta) + y * np.cos(theta))**2 # 4*a_i2 + a_j2
    gaussEnv = np.exp(- 1/(sigma**2*8.0) * gaussEnv)

    # Sinusoidal part of Gabor function
    cplxPart = spat_freq * (x * np.cos(theta) + y * np.sin(theta))
    cplxPart_cos = np.cos(cplxPart)
    cplxPart_sin = np.sin(cplxPart)

    z_cos = gaussEnv * cplxPart_cos
    z_sin = gaussEnv * cplxPart_sin
    
    return [z_cos, z_sin]

In [ ]:
def choosing_neighbors(i, num_kernel, num_row, num_col, radius):
    # (kernel, row, column) -> flatten
    N_e = num_kernel * num_row * num_col
    kernel_num = int(i / (num_row * num_col))
    row_num = int((i - kernel_num * num_row * num_col) / num_col)
    col_num = int((i - kernel_num * num_row * num_col - row_num * num_col))
    same_hypercolumn = np.arange(col_num + row_num * num_col, N_e, num_row * num_col)
    result = np.array([], dtype = int)
    
    for col in range(int(2 * radius + 1)):
        if (col_num - radius + col >= 0 and col_num - radius + col <= num_col-1):
            for row in range(int(2 * radius + 1)):
                if (row_num - radius + row >= 0 and row_num - radius + row <= num_row-1):
                    result = np.append(result, np.arange(col_num - radius + col + (row_num - radius + row) * num_col, N_e, num_row * num_col, dtype = int))
    return result

In [7]:
def choosing_inhibition_neighbors(i, num_kernel, num_row, num_col, radius_inhibition):
    # (kernel, row, column) -> flatten
    # including neuron i itself
    N_e = num_kernel * num_row * num_col
    kernel_num = int(i / (num_row * num_col))
    row_num = int((i - kernel_num * num_row * num_col) / num_col)
    col_num = int((i - kernel_num * num_row * num_col - row_num * num_col))
    same_hypercolumn = np.arange(col_num + row_num * num_col, N_e, num_row * num_col)
    result = np.array([], dtype = int)
    
    for col in range(int(2 * radius_inhibition + 1)):
        if (col_num - radius_inhibition + col >= 0 and col_num - radius_inhibition + col <= num_col-1):
            for row in range(int(2 * radius_inhibition + 1)):
                if (row_num - radius_inhibition + row >= 0 and row_num - radius_inhibition + row <= num_row-1):
                    result = np.append(result, kernel_num * num_row * num_col + col_num - radius_inhibition + col + \
                    (row_num - radius_inhibition + row) * num_col)
    return result 

In [ ]:
def initialization(num_layers, N_e, N_i):
    ue = []
    ui = []
    re = []
    ri = []
    duedt = []
    duidt = []
    dwrpdt = []
    
    for i in range(num_layers):
        ue.append(np.zeros(N_e[i]))
        re.append(np.zeros(N_e[i]))
        ui.append(np.zeros(N_i[i]))
        ri.append(np.zeros(N_i[i]))
        duedt.append(np.zeros(N_e[i]))
        duidt.append(np.zeros(N_i[i]))
        dwrpdt.append(np.zeros(N_e[i] * N_e[i]))
    
    return [ue, ui, re, ri, duedt, duidt, dwrpdt]

def initialization_between(num_layers, N_e):
    dwffdt = []
    dwfbdt = []
    
    for i in range(num_layers-1):
        dwffdt.append(np.zeros(N_e[i] * N_e[i+1]))
        dwfbdt.append(np.zeros(N_e[i] * N_e[i+1]))
    
    return [dwffdt, dwfbdt]

In [ ]:
def take_center(y, num_kernel, num_row, num_col, center = 3):
    # only take neurons in the center 3-by-3 hypercolumns
    # (kernel, row, column) -> flatten
    assert len(y) == num_kernel * num_row * num_col
    yr = np.reshape(y, (num_kernel, num_row * num_col))
    no_hyper = np.array([], dtype = int)
    for i in range(center):
        no_hyper = np.append(no_hyper, (num_row - center) // 2 * num_col + (num_col - center) // 2 + np.arange(center) + i * num_col)
    return yr[:, no_hyper].flatten()